## Ski Rental with Movie Lens and CitiBike
This notebook trains predictors for whether a user will watch more than b movies in the next month. 
We use these raw and calibrated predictors to compare the results of the ski-rental algorithms we propose in our paper.

In [ ]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme() 
# features
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_curve
from sklearn.neural_network import MLPRegressor


#calibration library
from crepes import WrapRegressor

# metrics
from sklearn.metrics import roc_auc_score
from sklearn import tree

import models as md
import utils as ut
import calibrator as cal
import skirental as sr

%load_ext autoreload 
%autoreload 2

In [ ]:
bike_df.keys()

### Section 0: Loading Data

In [ ]:
DATASET = 'Bike'

if DATASET == 'Movies': 
    movie_df = pd.read_csv('clean_data/movie_lens100k_monthly.csv')
    y = movie_df['movies_this_month'].values
    # Transform both feature sets
    X_clf = movie_df[['age', 'gender', 'occupation', 'zip', 'movies_watched']]
    X_reg = movie_df[['age', 'gender', 'movies_watched']]

    # Create separate DictVectorizers for each feature set
    vec_clf = DictVectorizer()
    vec_reg = DictVectorizer()

    X_post_clf = vec_clf.fit_transform(X_clf.to_dict('records')).toarray()
    X_post_reg = vec_reg.fit_transform(X_reg.to_dict('records')).toarray()
elif DATASET == 'Bike': 
    n=10000
    bike_df = pd.read_csv('clean_data/citibike-june2015.csv').dropna().sample(n)
    #june_bike_df = pd.read_csv('clean_data/citibike-june2015.csv').dropna().sample(n)
    
    #fig, axes = plt.subplots(2, 1, sharex=True)
    #sns.histplot(bike_df['tripduration'].values, ax=axes[0])
    #sns.histplot(june_bike_df['tripduration'].values, ax=axes[1])
    
    bike_df = bike_df[bike_df['start station name']!= bike_df['end station name']]

    #joint_df = ut.uniform_subsample_df(bike_df, 'tripduration', n_bins=10, samples_per_bin=1000) #.sample(n)
    #print(joint_df.shape)
    joint_df = bike_df
    # positive = bike_df[bike_df['tripduration'] >= b].sample(int(n/2))
    # negative = bike_df[bike_df['tripduration'] < b].sample(int(n/2))
    # joint_df = pd.concat([positive, negative])

    X = joint_df[['birth year', 'gender', 'usertype', 'start latitude rounded', 'start longitude rounded', 'end latitude rounded']] #, 'end longitude rounded', 'starthour']]
    y = joint_df['tripduration'].values

    

    # Create separate DictVectorizers for each feature set
    vec_clf = DictVectorizer()
    vec_reg = DictVectorizer()

    X_post_clf = vec_clf.fit_transform(X.to_dict('records')).toarray()
    X_post_reg = vec_reg.fit_transform(X.to_dict('records')).toarray()

# sns.histplot(y)
# quantiles = joint_df['tripduration'].quantile([0.25, 0.5, 0.75])
# for q in quantiles:
#     print(q)
#     plt.axvline(q, color='red', linestyle='--')
# plt.xlabel('target value')
# plt.title(f'Distribution of target values for {DATASET} dataset')

In [ ]:
X

### Section 1: Training & Calibrating Predictors
This section is to verify the achievable accuracy / perforamnce of predictors on this dataset

In [ ]:
# training calibrated regression model: 
i=0
X_train, X_test, y_train, y_test = train_test_split(
    X_post_reg, y, test_size=0.20, random_state=i)

X_train, X_valid, y_train, y_valid = train_test_split(
        X_train, y_train, test_size=0.20, random_state=i)

print(f"size train: {len(X_train)}, valid: {len(X_valid)} test: {len(X_test)} ")

for clf in md.reg_dict: 
    start_time = time.time()
    model = Pipeline(
                [
                    ("scalar", StandardScaler()),
                    ("reg", md.reg_dict[clf]())
                ]
            )
    model.fit(X_train, y_train)
    print(f"{clf}: {model.score(X_test, y_test)}")
    rf = WrapRegressor(model)
    rf.fit(X_train, y_train)
    rf.calibrate(X_valid, y_valid)
    for conf in [0.7, 0.8, 0.9, 0.95]: 
        conf_interval = rf.predict_int(X_test, confidence=conf)
        print(f'conf: {conf} interval size: {(conf_interval[:, 1] - conf_interval[:, 0]).mean()}')
    end_time = time.time()
    execution_time = end_time - start_time 
    print(f"Function took {execution_time} seconds to execute\n") 

In [ ]:
training_results = [] 
num_runs = 1
binary_clfs = {} 
calibrators = {} 
bins = 50 
temp_b = 600
for i in range(num_runs): 
        X_train, X_test, y_train, y_test = train_test_split(
            X_post_clf, y>temp_b, test_size=0.20, random_state=i)

        X_train, X_valid, y_train, y_valid = train_test_split(
                X_train, y_train, test_size=0.20, random_state=i)

        for name in md.clf_dict: 
            start_time = time.time()
            curr_clf = md.model_choice(name, X_train, y_train)
            curr_clf.fit(X_train, y_train)
            probs = curr_clf.predict_proba(X_test)[:, 1]
            _, _, ece_l1, ece_l2, alpha, = ut.expected_calibration_error(prob_true=np.asarray(y_test), 
                                                                 prob_pred=np.asarray(probs), 
                                                                 num_bins=bins)
            training_results.append({
                'clf': name, 
                'run': i, 
                'ece_l1': ece_l1, 
                'ece_l2': ece_l2, 
                'alpha': alpha,
                'accuracy': curr_clf.score(X_test, y_test), 
                'auc': roc_auc_score(y_test, probs),
                'calibrated': False
            })
            binary_clfs[name] = curr_clf

            calibrator = cal.BinningCalibrator(bins=bins)
            calibrator.calibrate(curr_clf.predict_proba(X_valid)[:, 1], y_valid)
            y_cal_probs = calibrator.transform(probs)
            _, _, ece_l1, ece_l2, alpha = ut.expected_calibration_error(prob_true=np.asarray(y_test),
                                                                 prob_pred=np.asarray(y_cal_probs),
                                                                 num_bins=bins)
            training_results.append({
                'clf': name, 
                'run': i, 
                'ece_l1': ece_l1, 
                'ece_l2': ece_l2, 
                'alpha': alpha,
                'accuracy': ((y_cal_probs > 0.5) ==  y_test).mean(), 
                'auc': roc_auc_score(y_test, y_cal_probs),
                'calibrated': True
            })
            calibrators[name] = calibrator
            end_time = time.time()
            execution_time = end_time - start_time 
            print(f"{name} took {execution_time} seconds to execute\n")

In [ ]:
training_results_df = pd.DataFrame(training_results)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 8))
metrics = ['ece_l1', 'ece_l2', 'accuracy', 'auc']
titles = ['ECE (L1)', 'ECE (L2)', 'Accuracy', 'AUC']
axes = axes.ravel()

# Create subplots
for idx, (metric, title) in enumerate(zip(metrics, titles)):
    sns.barplot(
        data=training_results_df,
        x='clf',
        y=metric,
        hue='calibrated',
        ax=axes[idx],
    )
    axes[idx].set_title(title)
    axes[idx].set_xlabel('Classifier')
    axes[idx].set_ylabel(title)
    # Rotate x-axis labels for better readability
    axes[idx].tick_params(axis='x', rotation=45)
    # Move legend to a better position
    axes[idx].legend(title='')

# Adjust layout
plt.tight_layout()
plt.show()

### Section 2: Comparing Ski Rental Algorithms

In [ ]:
CR_df = []
bins=100
num_runs = 5
conf = 0.9
debug_dict = {} 
for i in range(num_runs):
    indices = np.arange(len(X))
    # First split: separate out test set (20% of data)
    idx_temp, idx_test = train_test_split(
        indices, test_size=0.20, random_state=i
    )

    # Second split: split remaining data into train and validation (0.25 = 20% of original 80%)
    idx_train, idx_valid = train_test_split(
        idx_temp, test_size=0.20, random_state=i
    )

    # regression 
    X_train = X_post_reg[idx_train]
    X_valid = X_post_reg[idx_valid]
    X_test = X_post_reg[idx_test]

    y_train_cont = y[idx_train]
    y_valid_cont = y[idx_valid]
    y_test_cont = y[idx_test]

    # continuous Y
    # reg_model_name = md.get_best_regressor(X_train,
    #                                        y_train,
    #                                        X_valid,
    #                                        y_valid)
    reg_results_df = pd.DataFrame()
    reg_model_name = 'NN'
#     model = Pipeline(
#         [
#             ("scalar", StandardScaler()),
#             #(reg_model_name, md.reg_dict[reg_model_name]())
#             (reg_model_name, MLPRegressor(hidden_layer_sizes=(8,2),solver="sgd",alpha=0.01))

#         ]
#     )
    model = md.model_choice_regression(reg_model_name, X_train, y_train_cont)
    rf = WrapRegressor(model)
    rf.fit(X_train, y_train_cont)
    rf.calibrate(X_valid, y_valid_cont)

    conf_interval = rf.predict_int(X_test, confidence=conf)

    reg_results_df['reg_pred_cont'] = model.predict(X_test)
    reg_results_df['reg_low_cont'] = np.maximum(conf_interval[:, 0], 1e-5) # avoid negative numbers
    reg_results_df['reg_high_cont'] = conf_interval[:, 1]
    reg_results_df['y'] = y_test_cont

    print("mean conf interval size: ", (conf_interval[:10, 1] - conf_interval[:10, 0]).mean())
    
    for b in [200, 300, 400, 500, 600, 700, 800, 900, 1000]:
        #b+=1300
    #for b in [800, 900, 1000]:    
        results_df = pd.DataFrame()

        
        y_train = y_train_cont > b 
        y_valid = y_valid_cont > b 
        y_test = y_test_cont > b 
            
        # reg_model_name = md.get_best_regressor(X_train,
        #                                        y_train_reg,
        #                                        X_valid,
        #                                        y_valid_reg)
        model = md.model_choice_regression(reg_model_name, X_train, y_train)
#         model = Pipeline(
#             [
#                 ("scalar", StandardScaler()),
#                 #(reg_model_name, md.reg_dict[reg_model_name]())
#                 (reg_model_name, MLPRegressor(hidden_layer_sizes=(8,2),solver="sgd",alpha=0.01))

#             ]
#         )
        rf = WrapRegressor(model)
        rf.fit(X_train, y_train)
        rf.calibrate(X_valid, y_valid)

        conf_interval = rf.predict_int(X_test, confidence=conf)
            
        results_df['reg_pred'] = model.predict(X_test)
        results_df['reg_low'] = np.maximum(conf_interval[:, 0], 1e-3)
        results_df['reg_high'] = conf_interval[:, 1]

        
        results_df['y_bin'] = y_test
        results_df['delta'] = 1 - conf
        
        # clf_model_name = md.get_best_classifier(X_train,
        #                                        y_train,
        #                                        X_valid,
        #                                        y_valid)
        clf_model_name = 'NN'
        curr_clf = md.model_choice(clf_model_name, X_train, y_train)
        curr_clf.fit(X_train, y_train)
  
    
        # calibrate model
        probs = curr_clf.predict_proba(X_test)[:, 1]
        valid_probs = curr_clf.predict_proba(X_valid)[:, 1]
        
        # get opt threshold: 
        fpr, tpr, thresholds = roc_curve(y_valid, valid_probs)
        j_scores = tpr - fpr
        optimal_idx = np.argmax(j_scores)
        optimal_threshold = thresholds[optimal_idx]
        
        #ABALATION
        results_df['clf_pred'] =  curr_clf.predict_proba(X_test)[:, 1] >= optimal_threshold
        results_df['eps'] = 1 - (results_df['clf_pred'] == y_test).mean()
        
#         clf_model_name = 'NN'
#         curr_clf = md.model_choice(clf_model_name, X_train, y_train)
#         curr_clf.fit(X_train, y_train)
#         probs = curr_clf.predict_proba(X_test)[:, 1]
#         valid_probs = curr_clf.predict_proba(X_valid)[:, 1]
        
        ## END ABALATION 
        
        calibrator = cal.HistogramCalibrator(bins=bins)
        calibrator.calibrate(y_prob=valid_probs, 
                             y_true=y_valid, 
                            subsample=True)
        
        y_cal_probs = calibrator.transform(probs)
        
        valid_probs = calibrator.transform(valid_probs)
        _, _, _, _, alpha = ut.expected_calibration_error(prob_true=np.asarray(y_valid),
                                             prob_pred=np.asarray(valid_probs),
                                             num_bins=bins, 
                                             subsample=True)
    
        # results_df['clf_pred'] =  curr_clf.predict_proba(X_test)[:, 1] >= optimal_threshold
        # results_df['eps'] = 1 - (results_df['clf_pred'] == y_test).mean()
        # print(1 - (results_df['clf_pred'] == y_test).mean())
        #print(f'acc: {(results_df['clf_pred'] == y_test).mean()}, vs {curr_clf.score(X_test, y_test)}')
        
        results_df['clf_prob'] = y_cal_probs
        results_df['alpha'] = alpha
        results_df['b'] = b
        
        results_df = pd.concat([reg_results_df, results_df], axis=1)

        results_df['pip_cont_day'] = results_df.apply(
            lambda row: sr.pip_alg(u=row['reg_high_cont'],l=row['reg_low_cont'], delta=row['delta'],b=row['b']), axis=1)
        
        results_df['pip_day'] = results_df.apply(
            lambda row: sr.pip_alg(u=row['reg_high']*b,l=row['reg_low']*b, delta=row['delta'], b=row['b']), axis=1)
        #results_df['pip_day'] = results_df['pip_day']*b

        results_df['bin_day'] = results_df.apply(
            lambda row: sr.regular_ski_rental(pred=row['clf_pred'], eps=row['eps'],b=row['b']), axis=1
        )
        results_df['cal_day'] = results_df.apply(
            lambda row: sr.cal_ski_rental(pred=row['clf_prob'], alpha=row['alpha'], b=row['b']), axis=1
        )
        results_df['breakeven_day'] = b
        results_df['breakeven2_day'] = 2*b
          
        results_df['pip_CR'] = sr.get_CR_df(results_df, 'pip_day')
        results_df['pip_cont_CR'] = sr.get_CR_df(results_df, 'pip_cont_day')
        results_df['bin_CR'] = sr.get_CR_df(results_df, 'bin_day')
        results_df['breakeven_CR'] = sr.get_CR_df(results_df, 'breakeven_day')
        results_df['breakeven2_CR'] = sr.get_CR_df(results_df, 'breakeven2_day')
        results_df['cal_CR'] = sr.get_CR_df(results_df, 'cal_day')

        CR_summary = results_df[['cal_CR', 'bin_CR','pip_CR', 'pip_cont_CR', 'breakeven_CR', 'breakeven2_CR']].mean().to_dict() 
        CR_summary['b'] = b
        CR_summary['i'] = i
        CR_summary['opt_thresh'] = optimal_threshold
        CR_df.append(CR_summary)
        
        if i == 0: 
            debug_dict[b] = results_df.copy()

In [ ]:
CR_df = pd.DataFrame(CR_df)

In [ ]:
CR_df.groupby('b').mean()

In [ ]:
CR_df.to_csv("ski_rental_final_results.csv")

In [ ]:
# Melt the DataFrame
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes= axes.flatten() 
melted_df = pd.melt(CR_df, 
                    # Keep b and i as id variables
                    id_vars=['b', 'i'],
                    # Select the CR columns to melt
                    value_vars=['pip_CR', 'bin_CR', 'breakeven_CR', 'cal_CR'],
                    #value_vars=['pip_CR', 'bin_CR', 'breakeven_CR', 'cal_CR'],
                    # Name the new columns
                    var_name='method',
                    value_name='CR')

sns.histplot(y, ax=axes[0])
for q in [0.25, 0.5, 0.75]:
    val = np.quantile(y, q)
    axes[0].axvline(val, color='blue', linestyle='--')
    axes[0].text(val, axes[0].get_ylim()[1], f'{val}', 
                 rotation=90, va='top', ha='right')
axes[0].set_xlabel('target value')
axes[0].set_title(f'Distribution of target values for {DATASET} dataset')

sns.lineplot(data=melted_df,
             x='b',
             y='CR',
             hue='method',
             style='method',
             marker='o', 
            ax=axes[1])  # This will add 95% confidence intervals

axes[1].set_title(f'Comparison of CR Methods clf:{clf_model_name} reg:{reg_model_name} (OPT Threshold)')
plt.savefig(f'clf{clf_model_name}-reg{reg_model_name}_CR.pdf')
fig.tight_layout()

## Debugging 

In [ ]:
b=600
results_df = debug_dict[b]

In [ ]:
# debug pip 
row = results_df.iloc[9]
print(row[['pip_cont_CR', 'cal_CR', 'pip_CR']])
print('cal', row['cal_day'], row['clf_prob'], row['reg_high_cont'], row['y'])
print('cont', row['pip_cont_day'], row['reg_low_cont'], row['reg_high_cont'], row['y'])
print('disc', row['pip_day'], row['reg_low']*row['b'], row['reg_high']*row['b'], row['y'])
print(sr.pip_alg(u=row['reg_high_cont'], 
           l=row['reg_low_cont'], 
           delta=row['delta'], 
           b=row['b']))
print(sr.pip_alg(u=row['reg_high']*row['b'], 
           l=row['reg_low']*row['b'], 
           delta=row['delta'], 
           b=row['b']))

# CR 
sr.get_CR(buy=row['pip_day'],
            y=row['y'],
            b=row['b'])

In [ ]:
## compare df 
compare_CR='bin_CR'
compare_day = 'bin_day'
results_df['worse'] = results_df['cal_CR'] > results_df[compare_CR]
results_df['better'] = results_df['cal_CR'] < results_df[compare_CR]

In [ ]:
worst_df = results_df[results_df['worse'] == True]
worst_df.groupby(['cal_CR', 'clf_prob', compare_CR, 'y', 'cal_day', compare_day]).count()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))
axes= axes.flatten() 
sns.histplot(data=results_df, x='cal_day', y=compare_day, ax=axes[0]).set_title("buy day")
sns.histplot(data=results_df, x='cal_CR', y=compare_CR, ax=axes[1]).set_title("CR")
plt.suptitle(f"cal CR vs {compare_CR} for b={b}")